### Reinforcement Learning

In this notebook we demo the Q-learning algorithm, one of the fundamental algorithms of reinforcement learning. In reinforcement learning, an agent interacts with the environment, experiencing a series of rewards based on its actions. The agent seeks to maximize its rewards by developing a strategy that learns to choose appropriate actions in each state. 

<img src="https://raw.githubusercontent.com/YData123/sds265-fa21/main/demos/reinforcement-learning/rl_animation.gif" width=300>

<br>

We illustrate Q-learning on the taxicab problem formulated by Tom Dietterich in the paper "Hierarchical Reinforcement Learning with the MAXQ Value Function Decomposition", as developed in the code 
from [OpenAI gym](https://github.com/openai). Our presentation follows [this tutorial](https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/).

We'll need the OpenAI gym package. This can be installed as shown below. We'll display some simple graphics using `IPython.display`.

In [1]:
#!pip install gym
import gym
import numpy as np
from IPython.display import clear_output
from time import sleep


The environment is a simple grid, with some barriers inserted to make things more interesting. A taxicab drives around the environment, picking up and delivering a passenger at four locations. A graphic of the environment is shown below.

<img src="https://raw.githubusercontent.com/YData123/sds265-fa21/main/demos/reinforcement-learning/taxi-shot.png" width=200>

The "ascii art" rendition of this environment appears as shown in the following output cell:

In [2]:
env = gym.make("Taxi-v3").env
env.render()


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



As described in the documentation, 
"there are four designated locations in the grid world indicated by R(ed), G(reen), Y(ellow), and B(lue). When the episode starts, the taxi starts off at a random square and the passenger is at a random location. The taxi drives to the passenger's location, picks up the passenger, drives to the passenger's destination (another one of the four specified locations), and then drops off the passenger. Once the passenger is dropped off, the episode ends."

With 25 taxi positions, 4 possible locations of the passenger waiting, 
the case where the passenger is in the taxi, and 4 possible destination 
locations, the total number of states is $25\times 5 \times 4 = 500$.

The passenger location are coded as integers in the following way:
- 0: R(ed)
- 1: G(reen)
- 2: Y(ellow)
- 3: B(lue)
- 4: in taxi

And the destinations are coded as:
- 0: R(ed)
- 1: G(reen)
- 2: Y(ellow)
- 3: B(lue)

Finally, there are 6 possible actions:
- 0: move south
- 1: move north
- 2: move east
- 3: move west
- 4: pickup passenger
- 5: drop off passenger

There is a default per-step reward of -1,
and a reward of +20 for delivering the passenger. 
Carrying out a "pickup" or "drop-off" action illegally has
a reward of -10.

In the ascii art graphics, the following color schemes are used:
- blue: passenger
- magenta: destination
- yellow: empty taxi
- green: full taxi
- other letters (R, G, Y and B): locations for passengers and destinations

The state space is represented as a tuple in the following way:
    
    
    state = (taxi_row, taxi_col, passenger_location, destination)

In [3]:
# Some helper functions for visualizing the sequence of states and actions

def render(env, stat, action, reward):
    return {'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward}


def print_frames(frames, delay=.1):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(delay)
        


#### Exploring randomly

We first look at the obviously terrible strategy of selecting
a random action at each time step. 

In [4]:
env.s = 328  
epochs = 0
penalties, reward = 0, 0
frames = [] 

done = False
while not done:
    action = env.action_space.sample() # choose a random action
    state, reward, done, _ = env.step(action)

    if reward == -10:
        penalties += 1
    
    frames.append(render(env, state, action, reward))
    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 2257
Penalties incurred: 771


As can be seen, the taxi wanders around randomly, banging into walls, picking up and dropping off the passenger at wrong places, and incurring large negative rewards.

In [5]:
print_frames(frames[0:100], delay=.1)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)

Timestep: 100
State: 448
Action: 4
Reward: -10


### Using Q-learning

We'll now learn a strategy using the Q-learning algorithm. 
The Q-learning algorithm maintains a "quality" variable $Q(s,a)$ for taking action $a$ in state $s$. This is a measure of the cumulative
rewards obtained by the algorithm when it takes action $a$ in state $s$. 

The quality should not be assessed 
purely based on the reward the action has in the current time step.
Rather, we need to take into account the future rewards that the algorithm in driving toward the goal.

As described in lecture, the quality function is updated
as follows:

\begin{align*}
Q(s,a) \longleftarrow (1-\alpha)\,Q(s,a) + \alpha \left(
\mbox{reward}(s,a) + \gamma \max_{a'} Q(\mbox{next}(s,a), a')\right)
\end{align*}

Here $s$ is the current state. When action $a$ is taken, 
a reward $\mbox{reward}(s,a)$ is given. Then, the algorithm 
moves to a new state, denoted $\mbox{next}(s,a)$. For example, 
if the taxi is at location $(2,2)$ and takes the "West" action
($a=3$), then there is a reward of -1, and the taxi moves 
to the new location $(2,3)$. If the cab is empty, it remains empty, and if it contains the passenger, the passenger remains.

The cumulative future rewards of this action are given by the expression 
$\max_{a'} Q(\mbox{next}(s,a), a')$. These future rewards are discounted by a factor $\gamma < 1$. This trades off short-term against long-term rewards. When $\gamma$ is small, the short-term rewards are favored.

Note that this update can be rewritten as follows:

\begin{align*}
Q(s,a) \longleftarrow \,Q(s,a) + \alpha \left(
\mbox{reward}(s,a) + \gamma \max_{a'} Q(\mbox{next}(s,a), a') - Q(s,a)\right)
\end{align*}

This form suggests that it can be viewed as a gradient *ascent* algorithm, with $\alpha$ as the step size. In fact, it can be shown that with descreasing step size the algorithm will converge, just as for gradient descent. 

In [6]:
import random
from tqdm import tqdm

q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for _ in tqdm(np.arange(50000)):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, _ = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1


100%|██████████| 50000/50000 [00:22<00:00, 2212.57it/s]


In [7]:
total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.26
Average penalties per episode: 0.0


In [8]:
epochs = 0
penalties, reward = 0, 0

frames = [] # for animation
state = env.reset()
done = False

while not done:
    action = np.argmax(q_table[state])
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append(render(env, state, action, reward))
    epochs += 1
    
    
print(f"Timesteps taken: {epochs}")
print(f"Penalties incurred: {penalties}")

Timesteps taken: 11
Penalties incurred: 0


Now we see that the car behaves much more "intelligently," efficiently picking up and delivering the passenger to the desired destination.

In [9]:
print_frames(frames, delay=.1)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 11
State: 0
Action: 5
Reward: 20
